In [1]:
# syft absolute
import syft as sy
from syft.types.syft_object import SyftObject
from syft import ActionObject

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /home/teo/OpenMined/PySyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=b1ff819282d242538a61908c11ed8599 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=f07577a8850e42cea0216601e033f274 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
import inspect
from syft.service.code.user_code_service import UserCodeService
from syft.service.policy.policy import InputPolicy, OutputPolicy
from syft.service.request.request import Request, UserCodeStatusChange
from syft.service.code.user_code import UserCodeStatus, UserCode, UserCodeStatusCollection
from syft.store.linked_obj import LinkedObject
from syft.types.datetime import DateTime
from syft.types.uid import UID

def test():
    pass

request_id = UID()
user_code = UserCode(
    id = UID(),
    node_uid = node_low.python_node.id,
    user_verify_key = client_low.credentials.verify_key,
    raw_code = "",
    input_policy_type = InputPolicy,
    input_policy_init_kwargs = None,
    input_policy_state = b"",
    output_policy_type = OutputPolicy,
    output_policy_init_kwargs = None,
    output_policy_state = b"",
    parsed_code = '',
    service_func_name = '',
    unique_func_name = '',
    user_unique_func_name = '', 
    code_hash = '',
    signature = inspect.signature(test),
    status = UserCodeStatusCollection(status_dict={}),
    input_kwargs = []
)

linked_obj = LinkedObject(
    node_uid = node_low.python_node.id,
    service_type = UserCodeService,
    object_type = UserCode,
    object_uid = user_code.id,
)


low_request = Request(
    id=request_id,
    requesting_user_verify_key=client_low.credentials.verify_key,
    requesting_user_name="Teo",
    requesting_user_email= "teo@om.org",
    requesting_user_institution= "OM",
    approving_user_verify_key= client_low.credentials.verify_key,
    request_time=DateTime.now(),
    updated_at=DateTime.now(),
    node_uid=node_low.python_node.id,
    request_hash='a',
    changes=[UserCodeStatusChange(
            value=UserCodeStatus.PENDING,
            linked_obj=linked_obj,
        )],
    history=[],
)


In [5]:
# import json
# def request_to_json(request):
#     dict = {
#         'id': request.id,
#         "requesting_user_verify_key": ,
#         "requesting_user_name": request.requesting_user_name,
#         "requesting_user_email": request.requesting_user_email,
#         "requesting_user_institution": request.requesting_user_institution,
#         "approving_user_verify_key": ,
#         "request_time": ,
#         "updated_at": ,
#         "node_uid": ,
#         "request_hash": ,
#         "changes": ,
#         "history": ,
#     }
    
# def json_to_request(json):


In [6]:
high_request = Request(
    id=request_id,
    requesting_user_verify_key=client_low.credentials.verify_key,
    requesting_user_name="Teodor",
    requesting_user_email= "teodor@om.org",
    requesting_user_institution= "OpenMined",
    approving_user_verify_key= client_low.credentials.verify_key,
    request_time=DateTime.now(),
    updated_at=DateTime.now(),
    node_uid=node_low.python_node.id,
    request_hash='b',
    changes=[UserCodeStatusChange(
            value=UserCodeStatus.APPROVED,
            linked_obj=linked_obj,
        )],
    history=[],
)


In [7]:
from syft.client.syncing import get_diff_request, Diff
request_diff = Diff(
    low_obj=low_request,
    high_obj=high_request,
    obj_type=Request,
    merge_state="DIFF",
    diff_list=get_diff_request(low_request, high_request)
)


USING:  check_change


In [8]:
request_diff

syft.client.syncing.Diff

In [9]:
[request_diff]

[syft.client.syncing.Diff]

In [10]:
# assert False

# create datasets

In [11]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

  0%|          | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|██████████| 1/1 [00:00<00:00,  8.17it/s]


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [12]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  8.83it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [13]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///tmp/0c3efcced4fc4d29bb5bb647d02af6e0.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=729582e3492c4aa1aed81eca9ac035f3 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_7670: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_7670
4c3b69edd6de4ec695fa0431b15ba353
override True
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [14]:
from time import sleep
sleep(1)

In [15]:
client_low.code.request_code_execution(compute_mean)

SyftInfo: Node Landed!


syft.service.request.request.Request

In [16]:
request = client_low.requests[0]

In [17]:
code = client_low.code[0]

In [18]:
from syft.client.syncing import get_sync_state
state = get_sync_state(client_low, client_high) # between two nodes

In [19]:
state.diff_to_sync

[syft.client.syncing.Diff, syft.client.syncing.Diff]

In [20]:
# state.objs_to_sync
from syft.client.syncing import resolve
low_items_to_sync, high_items_to_sync = resolve(state)

```python
class Request:
  id: str = 42c59a14d5a946a28e589b0a84d3b8e1
  request_time: str = 2024-02-16 13:22:54
  updated_at: str = None
  status: str = RequestStatus.PENDING
  changes: str = ['Request to change <b>compute_mean</b> (Pool Id: <b>default-pool</b>) to permission <b>RequestStatus.APPROVED</b>. No nested requests']
  requesting_user_verify_key: str = 1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367

```

Do you approve moving this object from the HIGH side to the LOW side (approve/deny): 


```python
class UserCode
    id: UID = 89d137b823474f9ab95eb2aa7f0177d5
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()

```

Do you approve moving this object from the HIGH side to the LOW side (approve/deny): 


- What do we do with properties here?
  - just set them to None

In [21]:
def get_stash_for_item(item, node):
    services = list(node.python_node.service_path_map.values())

    all_stashes = dict()
    for serv in services:
        if (_stash := getattr(serv, "stash", None)) is not None:
            all_stashes[_stash.object_type] = _stash

    stash = all_stashes.get(type(item), None)
    return stash

In [22]:
def set_obj_ids(x, node_uid, verify_key):
    if hasattr(x, "__dict__") and isinstance(x, SyftObject):
        for prop, val in x.__dict__.items():
            if isinstance(val, (list, tuple)):
                for v in val:
                    set_obj_ids(v, node_uid, verify_key)
            elif isinstance(val, dict):
                for v in val.values():
                    set_obj_ids(v, node_uid, verify_key)
            else:
                set_obj_ids(val, node_uid, verify_key)
        x.syft_node_location=node_uid
        x.syft_client_verify_key=verify_key
        if hasattr(x, "node_uid"):
            x.node_uid=node_uid
                    
                    
    # TODO: hadnle nested collections
            

In [23]:
def create_actionobject(action_object, client):
    action_object = action_object.refresh_object()
    res = client.api.services.action.set(action_object)
    print("Created", res)

In [24]:
def sync_items(items, node, client):
    for item in items:
        
        if isinstance(item, ActionObject):
            create_actionobject(item, client)

        else:
            stash = get_stash_for_item(item, node)

            set_obj_ids(item, client.id, client.verify_key)

            res = stash.delete_by_uid(node.python_node.verify_key, item.id)

            res = stash.set(node.python_node.verify_key, item)
            if not res.is_ok():
                raise ValueError("")
            else:
                print("created", item)

In [25]:
sync_items(high_items_to_sync, node_high, client_high)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


In [26]:
state = get_sync_state(client_low, client_high)
state.diff_to_sync

USING:  check_change


[syft.client.syncing.Diff, syft.client.syncing.Diff]

# Run code high and sync back result

In [27]:
client_high._fetch_api(client_high.credentials)

In [28]:
data_high = client_high.datasets[0].assets[0]

In [29]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 60450da0734a488d85fe29a956129284
    status: JobStatus.CREATED
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [30]:
# wait for the result
job_high.wait()

89d137b823474f9ab95eb2aa7f0177d5
override True


```python
Pointer
```
17.0

In [31]:
job_high.result.get()

17.0

In [32]:
request_high = client_high.requests[0]
code = request_high.code
log = job_high._get_log_objs()

In [33]:
items_to_sync = [code, job_high.result, job_high, log]

In [34]:
state_2 = get_sync_state(client_low, client_high)

USING:  check_change


In [35]:
state_2.diff_to_sync

[syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff]

In [36]:
# items_to_sync = state_2.objs_to_sync()
# items_to_sync.append(job_high.result)
low_items_to_sync, high_items_to_sync = resolve(state_2)

```python
class Job:
    id: UID = 60450da0734a488d85fe29a956129284
    status: JobStatus.COMPLETED
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


```python
Pointer
```
17.0

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


```python
class SyftLog:
  id: str = 9203b3c524c64d87a5a1551a209d6045

```

Do you approve moving this object from the LOW side to the HIGH side (approve/deny): 


In [37]:
x = client_high.requests[0]

In [38]:
request_high.approve()

Approving request for domain test_h


"Cannot Modify Status as the Domain's data is not included in the request"

In [39]:
sync_items(low_items_to_sync, node_low, client_low)

created syft.service.job.job_stash.Job
Created SyftError: Arg: Could not find item with uid db1b84210b044cf193b7025fc922fe44, "db1b84210b044cf193b7025fc922fe44 not in <class 'syft.store.sqlite_document_store.SQLiteBackingStore'>" must be Union not str
created syft.service.log.log.SyftLog


In [40]:
# low_node_state = client_low.get_sync_state() -> just a table
# high_node_state = client_high.get_sync_state()
# (low_new_state, high_new_state) = resolve_state(low_node_state, high_node_state)

In [41]:
state_3 = get_sync_state(client_low, client_high)

USING:  check_change


In [42]:
state_3.diff_to_sync

[syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff,
 syft.client.syncing.Diff]

In [43]:
from syft.client.syncing import check_change, check_linked_obj

In [44]:
low_change = state_3.diff_to_sync[0].low_obj.changes[0]

In [45]:
high_change = state_3.diff_to_sync[0].high_obj.changes[0]

In [46]:
check_change(low_change, high_change)

True

In [47]:
low_change.value

<UserCodeStatus.APPROVED: 'approved'>

In [48]:
high_change.value == low_change.value

True

In [49]:
high_change.linked_obj == low_change.linked_obj

False

In [50]:
high_change.match_type == low_change.match_type

True

In [51]:
high_change.linked_obj.service_type

syft.service.code.user_code_service.UserCodeService

In [52]:
low_change.linked_obj.service_type

syft.service.code.user_code_service.UserCodeService

# Run code low

In [53]:
res_low = client_low.code.compute_mean(data=data_low)

89d137b823474f9ab95eb2aa7f0177d5
override True


In [54]:
# we need to sync the usercode itself


In [55]:
res_low.get()

7.0

In [56]:
private_high.mean()

17.0

In [57]:
job_low = client_low.code.compute_mean(data=data_low, blocking=False)

89d137b823474f9ab95eb2aa7f0177d5


SyftWarning: There are existing jobs for this user code, returning the latest one

In [58]:
job_low

```python
class Job:
    id: UID = 60450da0734a488d85fe29a956129284
    status: JobStatus.COMPLETED
    has_parent: False
    result: 17.0
    logs:

0 
JOB COMPLETED
    
```

In [59]:
job_low.result

```python
Pointer
```
17.0